### Raw Data Download Notebook

*Important Note:* This notebook runs using a different environment than this project's computational notebook. ObsPy is required, along with numpy and matplotlib

Zali et al. 2024 pulls horizontal seismic data from a single horizontal component from a single station, demeaned and detrended but not converted from instrument response. The data is pulled from March 12 to June 24, ~104 days, beginning 7 days before the Geldingadalir eruption began. The data is also downsampled to the minimum sampling rate necessary to observe the local volcanic tremor.

------------------

This notebook will download 100 days of data, beginning on March 1st, 2005 and ending June 9th, 2005, from sensor CC.STD of the Cascade Chain Volcano Monitoring network, located on Mount St. Helens on Studebaker Ridge. Mount St. Helens had a series of eruptions of ash and steam between 2004 and 2008, but this will focus on the second explosive eruption on March 8th, 2005 that was preceeded by seismic activity. https://www.usgs.gov/volcanoes/mount-st.-helens/science/2004-2008-renewed-volcanic-activity 

In [2]:
!pip install obspy

In [1]:
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client
client = Client('IRIS')
import numpy as np
import os

In [2]:
def process(st, buffer, freq, max_target_frequency): #freq is the original sampling frequency
    st.merge(fill_value='interpolate')
    tr = st[0].copy()

### Data Description

The seismic data is downloaded from the IRIS/Earthscope Database using ObsPy. The data downloads as a trace, an array of data with attached metadata, which is then packaged into a stream, which can contain multiple traces. The daily seismic data records are then saved as mseed files, which preserve this data+metadata structure, but requires ObsPy or other specialized software to open.

In [3]:
#creating variables to download data
starttime = utc('2005-03-01T00:00:00')
endtime = starttime + 100 * (60*60*24)

#also add a buffer to both ends to chop off once the data has been filtered
#and downsampled, kind of arbitrary length, 5% of a day (default ObsPy taper length)
buffer = 60*60*24*0.05 #seconds

net = 'CC'
sta = 'STD'
loc = '*' #wildcard, generally don't care about location code
cha = 'BHN' #horizontal component, as used in Zali et al

#create folder for numpy streams to go into and initialize filepath
!mkdir data
!mkdir data/raw
filepath = os.getcwd() + '/data/raw/'

#create arrays to save dates
dates = np.array([])

print(f'Downloading data from {starttime} to {endtime}')
#download the data piecemeal, here by day
for day in range(100):
    tr_length = 24*60*60

    try:
        #actually downloading
        st = client.get_waveforms(network=net,
                     station=sta,
                     location=loc,
                     channel=cha,
                     starttime=starttime-buffer,
                     endtime=starttime+buffer+tr_length)

        #instrument sampling rate (hz)
        freq = st[0].stats.sampling_rate

        #merge traces within stream, linearly interpolating any gaps
        st.merge(fill_value='interpolate')
    
        #generate filename, day number in front for convenience of reading in
        name = str(day+1)+'_sthelens.mseed'
    
        #save data as mseed, standard for storing seismic data. Preserves metadata and time series info
        st.write(filepath+name, format='MSEED')

        #adding date
        dates = np.append(dates, starttime.date)

    except FDSNNoDataException:
        print(f'No data available for {starttime.date()}')
    

        starttime += tr_length


#save dates list for future use
np.save(filepath+'date_list.csv', dates)

print(f'data download complete, saved to {filepath}')



mkdir: data: File exists
mkdir: data/raw: File exists
data download complete, saved to /Users/KatarzynaPerks/Documents/GitHub/Perks_Geldingadalir/notebooks/data/raw/


### Data Modalities and Formats

#### Data Modalities
The dataset consists of seismic data collected from a single horizontal component (BHN) of the CC.STD station located on Mount St. Helens. The data is recorded continuously over a period of 100 days, capturing the seismic activity before, during, and after the steam and ash eruption.

#### Data Formats
1. **MSEED (Mini-SEED) Files**:
    - The seismic data is stored in Mini-SEED format, which is a compact binary format used for storing time series data. Each file contains a day's worth of seismic data, including metadata such as the sampling rate and station information.
    - Example file name: `1_sthelens.mseed`, `2_sthelens.mseed`, ..., `100_sthelens.mseed`.
2. **Numpy Arrays**:
    - The dates corresponding to each day's seismic data are stored in a numpy array and saved as a CSV file (`date_list.csv`). This array helps in mapping the MSEED files to their respective dates.
    - Example: `dates = np.array([datetime.date(2005, 3, 1), datetime.date(2005, 3, 2), ...])`.

#### Data Processing
- The raw seismic data is downloaded using the ObsPy library from the IRIS/Earthscope Database.
- The data is merged and interpolated to fill any gaps, ensuring a continuous time series.
- The processed data is saved in the MSEED format, preserving both the time series and metadata.

This structured approach ensures that the seismic data is well-organized and easily accessible for further analysis and processing.

In [4]:
# display some data from the mseed file
print(f'Example data from {name}')
print(st[0].stats)
print()

# display the numpy array
print(f'Example data from {name}')
print(st[0].data)
print()

Example data from 100_sthelens.mseed
               network: CC
               station: STD
              location: 
               channel: BHN
             starttime: 2005-02-28T22:48:00.000000Z
               endtime: 2005-03-02T01:12:00.000000Z
         sampling_rate: 50.0
                 delta: 0.02
                  npts: 4752001
                 calib: 1.0
_fdsnws_dataselect_url: http://service.iris.edu/fdsnws/dataselect/1/query
               _format: MSEED
                 mseed: AttribDict({'dataquality': 'M', 'number_of_records': 3282, 'encoding': 'STEIM1', 'byteorder': '>', 'record_length': 512, 'filesize': 11978752})
            processing: ['ObsPy 1.4.1: trim(endtime=UTCDateTime(2005, 3, 2, 1, 12)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2005, 2, 28, 22, 48))']

Example data from 100_sthelens.mseed
[353322 349615 345409 ... 346392 340573 335078]

